In [1]:
from typing import Callable

# Core dependencies
import jax
import jax.numpy as jnp
import equinox as eqx
import optax

# pcax
import pcax as px
import pcax.predictive_coding as pxc
import pcax.nn as pxnn
import pcax.functional as pxf
import pcax.utils as pxu

import torchvision
#import torchvision.transforms as transforms
from torchvision import datasets, transforms
import torch
import matplotlib.pyplot as plt
import numpy as np
from collections import namedtuple

2024-05-25 21:39:35.951874: W external/xla/xla/service/gpu/nvptx_compiler.cc:760] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.5.40). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [2]:
# Model definition
class TwoLayerNN(pxc.EnergyModule):
    def __init__(self, input_dim: int, hidden_dim: int, output_dim: int, act_fn: Callable[[jax.Array], jax.Array]) -> None:
        super().__init__()
        self.act_fn = px.static(act_fn)
        self.layers = [
            pxnn.Linear(input_dim, hidden_dim),
            pxnn.Linear(hidden_dim, output_dim)
        ]

        # create a glorot uniform initializer
        initializer = jax.nn.initializers.glorot_uniform()
        # now apply glorot uniform initialization to the weights only
        # the basic syntax is: model.layers[i].nn.weight.set(initializer(key, model.layers[i].nn.weight.shape))
        for l in self.layers:
            l.nn.weight.set(initializer(px.RKG(), l.nn.weight.shape))
        
        self.vodes = [
            pxc.Vode((hidden_dim,)),
            pxc.Vode((output_dim,), pxc.ce_energy)
        ]
        self.vodes[-1].h.frozen = True

    def __call__(self, x, y):
        for v, l in zip(self.vodes[:-1], self.layers[:-1]):
            x = v(self.act_fn(l(x)))
        x = self.vodes[-1](self.layers[-1](x))
        if y is not None:
            self.vodes[-1].set("h", y)
        return self.vodes[-1].get("u")

In [3]:
Dataset = namedtuple("Dataset", ["train_loader", "val_loader", "test_loader"])

# This is a simple collate function that stacks numpy arrays used to interface
# the PyTorch dataloader with JAX. In the future we hope to provide custom dataloaders
# that are independent of PyTorch.
def numpy_collate(batch):
    if isinstance(batch[0], np.ndarray):
        return np.stack(batch)
    elif isinstance(batch[0], (tuple, list)):
        transposed = zip(*batch)
        return [numpy_collate(samples) for samples in transposed]
    else:
        return np.array(batch)

# The dataloader assumes cuda is being used, as such it sets 'pin_memory = True' and
# 'prefetch_factor = 2'. Note that the batch size should be constant during training, so
# we set 'drop_last = True' to avoid having to deal with variable batch sizes.
class TorchDataloader(torch.utils.data.DataLoader):
    def __init__(
        self,
        dataset,
        batch_size=1,
        shuffle=None,
        sampler=None,
        batch_sampler=None,
        num_workers=16,
        pin_memory=True,
        timeout=0,
        worker_init_fn=None,
        persistent_workers=True,
        prefetch_factor=2,
    ):
        super(self.__class__, self).__init__(
            dataset,
            batch_size=batch_size,
            shuffle=shuffle,
            sampler=sampler,
            batch_sampler=batch_sampler,
            num_workers=num_workers,
            collate_fn=numpy_collate,
            pin_memory=pin_memory,
            drop_last=True if batch_sampler is None else None,
            timeout=timeout,
            worker_init_fn=worker_init_fn,
            persistent_workers=persistent_workers,
            prefetch_factor=prefetch_factor,
        )

# Function to add noise to the labels in the dataset
def add_label_noise(dataset, noise_level=0.2):
    targets = np.array(dataset.targets)
    num_classes = len(np.unique(targets))
    num_noisy = int(noise_level * len(targets))
    noisy_indices = np.random.choice(len(targets), num_noisy, replace=False)

    for idx in noisy_indices:
        original_label = targets[idx]
        new_label = original_label
        while new_label == original_label:
            new_label = np.random.randint(0, num_classes)
        targets[idx] = new_label

    dataset.targets = torch.tensor(targets)
    return dataset

# Function to get the dataloaders
def get_dataloaders(dataset_name, train_subset_size, batch_size, noise_level=0.2):
    if dataset_name.lower() == "mnist":
        ds = datasets.MNIST
    else:
        raise NotImplementedError(f"Dataset {dataset_name} isn't available")

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)),
        transforms.Lambda(lambda x: x.view(-1).numpy())  # Flatten the image to a vector
    ])

    train_set = ds(root='./data', download=True, train=True, transform=transform)
    train_set = add_label_noise(train_set, noise_level=noise_level)

    val_subset_size = int(0.2 * train_subset_size)
    random_train_indices = np.random.choice(len(train_set), size=train_subset_size, replace=False)
    remaining_indices = np.setdiff1d(np.arange(len(train_set)), random_train_indices)
    random_val_indices = np.random.choice(remaining_indices, size=val_subset_size, replace=False)

    train_loader = TorchDataloader(
        train_set, batch_size=batch_size, num_workers=16,
        sampler=torch.utils.data.sampler.SubsetRandomSampler(random_train_indices))
    val_loader = TorchDataloader(
        train_set, batch_size=batch_size, num_workers=16,
        sampler=torch.utils.data.sampler.SubsetRandomSampler(random_val_indices))

    test_set = ds(root='./data', download=True, train=False, transform=transform)
    test_loader = TorchDataloader(
        test_set, batch_size=batch_size, shuffle=False, num_workers=16)

    return Dataset(train_loader=train_loader, val_loader=val_loader, test_loader=test_loader)

In [4]:
"""
# This is a simple collate function that stacks numpy arrays used to interface
# the PyTorch dataloader with JAX. In the future we hope to provide custom dataloaders
# that are independent of PyTorch.
def numpy_collate(batch):
    if isinstance(batch[0], np.ndarray):
        return np.stack(batch)
    elif isinstance(batch[0], (tuple, list)):
        transposed = zip(*batch)
        return [numpy_collate(samples) for samples in transposed]
    else:
        return np.array(batch)

# The dataloader assumes cuda is being used, as such it sets 'pin_memory = True' and
# 'prefetch_factor = 2'. Note that the batch size should be constant during training, so
# we set 'drop_last = True' to avoid having to deal with variable batch sizes.
class TorchDataloader(torch.utils.data.DataLoader):
    def __init__(
        self,
        dataset,
        batch_size=1,
        shuffle=None,
        sampler=None,
        batch_sampler=None,
        num_workers=16,
        pin_memory=True,
        timeout=0,
        worker_init_fn=None,
        persistent_workers=True,
        prefetch_factor=2,
    ):
        super(self.__class__, self).__init__(
            dataset,
            batch_size=batch_size,
            shuffle=shuffle,
            sampler=sampler,
            batch_sampler=batch_sampler,
            num_workers=num_workers,
            collate_fn=numpy_collate,
            pin_memory=pin_memory,
            drop_last=True if batch_sampler is None else None,
            timeout=timeout,
            worker_init_fn=worker_init_fn,
            persistent_workers=persistent_workers,
            prefetch_factor=prefetch_factor,
        )

# This function returns the MNIST dataloaders for training and testing.
def get_dataloaders(batch_size: int):
    t = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)),
        transforms.Lambda(lambda x: x.view(-1).numpy())  # Flatten the image to a vector
    ])

    train_dataset = torchvision.datasets.MNIST(
        "~/tmp/mnist/",
        transform=t,
        download=True,
        train=True,
    )

    train_dataloader = TorchDataloader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=16,
    )

    test_dataset = torchvision.datasets.MNIST(
        "~/tmp/mnist/",
        transform=t,
        download=True,
        train=False,
    )
        
    test_dataloader = TorchDataloader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=16,
    )

    return train_dataloader, test_dataloader
"""

'\n# This is a simple collate function that stacks numpy arrays used to interface\n# the PyTorch dataloader with JAX. In the future we hope to provide custom dataloaders\n# that are independent of PyTorch.\ndef numpy_collate(batch):\n    if isinstance(batch[0], np.ndarray):\n        return np.stack(batch)\n    elif isinstance(batch[0], (tuple, list)):\n        transposed = zip(*batch)\n        return [numpy_collate(samples) for samples in transposed]\n    else:\n        return np.array(batch)\n\n# The dataloader assumes cuda is being used, as such it sets \'pin_memory = True\' and\n# \'prefetch_factor = 2\'. Note that the batch size should be constant during training, so\n# we set \'drop_last = True\' to avoid having to deal with variable batch sizes.\nclass TorchDataloader(torch.utils.data.DataLoader):\n    def __init__(\n        self,\n        dataset,\n        batch_size=1,\n        shuffle=None,\n        sampler=None,\n        batch_sampler=None,\n        num_workers=16,\n        p

In [5]:
# Training and evaluation functions
@pxf.vmap(pxu.Mask(pxc.VodeParam | pxc.VodeParam.Cache, (None, 0)), in_axes=(0, 0), out_axes=0)
def forward(x, y, *, model: TwoLayerNN):
    return model(x, y)

@pxf.vmap(pxu.Mask(pxc.VodeParam | pxc.VodeParam.Cache, (None, 0)), in_axes=(0,), out_axes=(None, 0), axis_name="batch")
def energy(x, *, model: TwoLayerNN):
    y_ = model(x, None)
    return jax.lax.pmean(model.energy().sum(), "batch"), y_

@pxf.jit(static_argnums=0)
def train_on_batch(T: int, x: jax.Array, y: jax.Array, *, model: TwoLayerNN, optim_w: pxu.Optim, optim_h: pxu.Optim):
    model.train()
    with pxu.step(model, pxc.STATUS.INIT, clear_params=pxc.VodeParam.Cache):
        forward(x, y, model=model)

    for _ in range(T):
        with pxu.step(model, clear_params=pxc.VodeParam.Cache):
            _, g = pxf.value_and_grad(pxu.Mask(pxu.m(pxc.VodeParam).has_not(frozen=True), [False, True]), has_aux=True)(energy)(x, model=model)
        optim_h.step(model, g["model"], True)

    with pxu.step(model, clear_params=pxc.VodeParam.Cache):
        _, g = pxf.value_and_grad(pxu.Mask(pxnn.LayerParam, [False, True]), has_aux=True)(energy)(x, model=model)
    optim_w.step(model, g["model"])

@pxf.jit()
def eval_on_batch(x: jax.Array, y: jax.Array, *, model: TwoLayerNN):
    model.eval()
    with pxu.step(model, pxc.STATUS.INIT, clear_params=pxc.VodeParam.Cache):
        y_ = forward(x, None, model=model).argmax(axis=-1)
    return (y_ == y).mean(), y_

def train(dl, T, *, model: TwoLayerNN, optim_w: pxu.Optim, optim_h: pxu.Optim):
    for x, y in dl:
        #print(f"x shape: {x.shape}, y shape: {y.shape}")  # Debugging line
        train_on_batch(T, x, jax.nn.one_hot(y, 10), model=model, optim_w=optim_w, optim_h=optim_h)

def eval(dl, *, model: TwoLayerNN):
    acc = []
    ys_ = []
    for x, y in dl:
        a, y_ = eval_on_batch(x, y, model=model)
        acc.append(a)
        ys_.append(y_)
    return np.mean(acc), np.concatenate(ys_)

In [6]:
# Model training and evaluation
batch_size = 128
nm_epochs = 500
model = TwoLayerNN(input_dim=784, hidden_dim=128, output_dim=10, act_fn=jax.nn.relu)

# Initialize the model and optimizers
with pxu.step(model, pxc.STATUS.INIT, clear_params=pxc.VodeParam.Cache):
    forward(jax.numpy.zeros((batch_size, 784)), None, model=model)
    optim_h = pxu.Optim(optax.sgd(0.1), pxu.Mask(pxc.VodeParam)(model))
    optim_w = pxu.Optim(optax.sgd(0.01, momentum=0.95), pxu.Mask(pxnn.LayerParam)(model))

# Assuming dataset is the namedtuple with the dataloaders
dataset = get_dataloaders("mnist", train_subset_size=4000, batch_size=batch_size, noise_level=0.0)
# Check the sizes of the datasets
print(f"Training set: {len(dataset.train_loader.sampler)} samples")
print(f"Validation set: {len(dataset.val_loader.sampler)} samples")
print(f"Test set: {len(dataset.test_loader.dataset)} samples")

Training set: 4000 samples
Validation set: 800 samples
Test set: 10000 samples


In [9]:
for e in range(nm_epochs):
    train(dataset.train_loader, T=10, model=model, optim_w=optim_w, optim_h=optim_h)
    # show the training accuracy, validation accuracy, and test accuracy all in one line together with the epoch number
    a_train, _ = eval(dataset.train_loader, model=model)
    a_test, _ = eval(dataset.test_loader, model=model)
    a_val, _ = eval(dataset.val_loader, model=model)

    print(f"Epoch {e + 1}/{nm_epochs} - Training Accuracy: {a_train * 100:.2f}% - Validation Accuracy: {a_val * 100:.2f}% - Test Accuracy: {a_test * 100:.2f}%")

Epoch 1/500 - Training Accuracy: 100.00% - Validation Accuracy: 93.62% - Test Accuracy: 94.04%
Epoch 2/500 - Training Accuracy: 100.00% - Validation Accuracy: 93.36% - Test Accuracy: 94.04%
Epoch 3/500 - Training Accuracy: 100.00% - Validation Accuracy: 93.36% - Test Accuracy: 94.04%
Epoch 4/500 - Training Accuracy: 100.00% - Validation Accuracy: 93.49% - Test Accuracy: 94.04%
Epoch 5/500 - Training Accuracy: 100.00% - Validation Accuracy: 93.10% - Test Accuracy: 94.04%
Epoch 6/500 - Training Accuracy: 100.00% - Validation Accuracy: 93.10% - Test Accuracy: 94.04%
Epoch 7/500 - Training Accuracy: 100.00% - Validation Accuracy: 93.36% - Test Accuracy: 94.04%
Epoch 8/500 - Training Accuracy: 100.00% - Validation Accuracy: 93.36% - Test Accuracy: 94.04%
Epoch 9/500 - Training Accuracy: 100.00% - Validation Accuracy: 93.88% - Test Accuracy: 94.04%
Epoch 10/500 - Training Accuracy: 100.00% - Validation Accuracy: 93.23% - Test Accuracy: 94.04%
Epoch 11/500 - Training Accuracy: 100.00% - Valid

In [ ]:
print(model)

In [ ]:
initializer = jax.nn.initializers.glorot_uniform()

In [ ]:
# show the mean of weights of model.layers[0].nn.weight before initialization
model.layers[0].nn.weight.get().mean()

In [ ]:
model.layers[0].nn.weight.set(initializer(px.RKG(), model.layers[0].nn.weight.shape))

In [ ]:
# show the weights mean of model.layers[0].nn.weight after initialization
model.layers[0].nn.weight.get().mean()